In [5]:
# import neccessary dependencies
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score

# Import the data

In [79]:
#training data
train = pd.read_csv(r"C:\Users\Admin\OneDrive\Documents\Webdev\hatespeech\train.csv", nrows=5000)

#testing data
test = pd.read_csv(r"C:\Users\Admin\OneDrive\Documents\Webdev\hatespeech\test.csv", nrows=5000)


# Clean the data of null values

In [7]:
# view data
train.head(20)

# for chunk in train:
#   display(chunk.head())
#   break

,text,label
0,Inaasahan na ni Vice President Jejomar Binay n...,0
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1
2,Salamat sa walang sawang suporta ng mga taga m...,0
3,@rapplerdotcom putangina mo binay TAKBO PA,1
4,"Binay with selective amnesia, forgetting about...",0
5,It doesn't matter whoever won between Duterte ...,0
6,Nognog? Pero nognog din ang nag malasakit? Wtf...,1
7,#OnlyB1nay ?? #FB https://t.co/QEQnsK67Gm,1
8,What Abi Binay said on running for Makati mayo...,0
9,Srsly. How can Binay do away with no tax for t...,1


In [8]:
# drop rows with null values
train.dropna()

,text,label
0,Inaasahan na ni Vice President Jejomar Binay n...,0
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1
2,Salamat sa walang sawang suporta ng mga taga m...,0
3,@rapplerdotcom putangina mo binay TAKBO PA,1
4,"Binay with selective amnesia, forgetting about...",0
...,...,...
9995,"Mar Roxas on the rise, w/ momentum, machinery,...",0
9996,@chelseapailmao,1
9997,"Alan Cayetano 'confirms' Palace, Roxas, Poe be...",0
9998,Mas nakakainis ad ni Mar kaysa kay Binay.,1


In [9]:
#check for null values in train
train.isnull().sum()

text     0
label    0
dtype: int64

In [10]:
# check for 0 values in train
sum(train["label"] == 0)

5340

In [11]:
# check for 1 values in train
sum(train["label"] == 1)

4660

In [12]:
# doucke check for null values
train.head(20)

,text,label
0,Inaasahan na ni Vice President Jejomar Binay n...,0
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1
2,Salamat sa walang sawang suporta ng mga taga m...,0
3,@rapplerdotcom putangina mo binay TAKBO PA,1
4,"Binay with selective amnesia, forgetting about...",0
5,It doesn't matter whoever won between Duterte ...,0
6,Nognog? Pero nognog din ang nag malasakit? Wtf...,1
7,#OnlyB1nay ?? #FB https://t.co/QEQnsK67Gm,1
8,What Abi Binay said on running for Makati mayo...,0
9,Srsly. How can Binay do away with no tax for t...,1


In [13]:
# drop null values in test
test.dropna()

,text,label
0,Unshaded votes and votes for Mayor Duterte goe...,1
1,#NoMoreChance https://t.co/msaaUGv0bS,1
2,@itsmanj well there's other good choices like ...,0
3,Nognog. Pandak. Laki sa hirap. Pero corrupt. Y...,1
4,Ex-Binay aide turns tables on Mercado | https:...,0
...,...,...
4227,@Ohohnikk @raachelleele kay nancy binay gagand...,1
4228,RT @chin_tessane: @ImangKuba @Senyora hahahaha...,1
4229,#onlybinay,0
4230,Trend Alert: 'Abby Binay'. More trends at http...,0


In [14]:
test.isnull().sum()

text     0
label    0
dtype: int64

In [15]:
test.head(20)

,text,label
0,Unshaded votes and votes for Mayor Duterte goe...,1
1,#NoMoreChance https://t.co/msaaUGv0bS,1
2,@itsmanj well there's other good choices like ...,0
3,Nognog. Pandak. Laki sa hirap. Pero corrupt. Y...,1
4,Ex-Binay aide turns tables on Mercado | https:...,0
5,2015/10/15 11:37 PHT,0
6,"Binay in Isabela radio interview: ‘Yong iba, ...",0
7,Poor Binay ?? https://t.co/In9yRo5QXI,1
8,RT @sizzlingbubbles: Juskopo! Gugustuhin kong ...,1
9,"ISA LANG MASASABI KO SA AD NI BINAY, P U T A. ...",1


In [16]:
train_shape = train.shape
print(train_shape)

# 10000 train data is available

(10000, 2)


In [14]:
test_shape = test.shape
print(test_shape)

#4232 test data is available

(4232, 2)


# Clean the data of unwated Text and Characters

In [18]:
# remove special characters using the regular expression library

import re

#set up punctuations we want to be replaced

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [19]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [20]:
# clean training data
train_tweet = clean_tweets(train["text"])
train_tweet = pd.DataFrame(train_tweet)

In [21]:
# append cleaned tweets to the training data
train["clean_tweet"] = train_tweet

# compare the cleaned and uncleaned tweets
train.head(50)

,text,label,clean_tweet
0,Inaasahan na ni Vice President Jejomar Binay n...,0,inaasahan na ni vice president jejomar binay n...
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1,mar roxas tang ina tuwid na daan daw eh sya n...
2,Salamat sa walang sawang suporta ng mga taga m...,0,salamat sa walang sawang suporta ng mga taga m...
3,@rapplerdotcom putangina mo binay TAKBO PA,1,putangina mo binay takbo pa
4,"Binay with selective amnesia, forgetting about...",0,binay with selective amnesia forgetting about ...
5,It doesn't matter whoever won between Duterte ...,0,it doesnt matter whoever won between duterte &...
6,Nognog? Pero nognog din ang nag malasakit? Wtf...,1,nognog pero nognog din ang nag malasakit wtf t...
7,#OnlyB1nay ?? #FB https://t.co/QEQnsK67Gm,1,
8,What Abi Binay said on running for Makati mayo...,0,what abi binay said on running for makati mayor
9,Srsly. How can Binay do away with no tax for t...,1,srsly how can binay do away with no tax for th...


In [22]:
sum(train['clean_tweet'] == '')

518

In [23]:
train.head(60)

,text,label,clean_tweet
0,Inaasahan na ni Vice President Jejomar Binay n...,0,inaasahan na ni vice president jejomar binay n...
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1,mar roxas tang ina tuwid na daan daw eh sya n...
2,Salamat sa walang sawang suporta ng mga taga m...,0,salamat sa walang sawang suporta ng mga taga m...
3,@rapplerdotcom putangina mo binay TAKBO PA,1,putangina mo binay takbo pa
4,"Binay with selective amnesia, forgetting about...",0,binay with selective amnesia forgetting about ...
5,It doesn't matter whoever won between Duterte ...,0,it doesnt matter whoever won between duterte &...
6,Nognog? Pero nognog din ang nag malasakit? Wtf...,1,nognog pero nognog din ang nag malasakit wtf t...
7,#OnlyB1nay ?? #FB https://t.co/QEQnsK67Gm,1,
8,What Abi Binay said on running for Makati mayo...,0,what abi binay said on running for makati mayor
9,Srsly. How can Binay do away with no tax for t...,1,srsly how can binay do away with no tax for th...


In [24]:
train.tail(60)

#now problem is since we cleaned the data, there are rows that have empty spaces, we need to remove the rows with empty
#values to avoid problems with training

,text,label,clean_tweet
9940,Masamang espiritu sana ang sumanib sayo! #Lune...,1,masamang espiritu sana ang sumanib sayo
9941,"""Kay Binay, gaganda ang buhay; oh oh oh oh oh....",0,kay binay gaganda ang buhay oh oh oh oh oh
9942,Please Wag Lang Poe Si Roxas! #Halalan2016 #Lo...,0,please wag lang poe si roxas feeling happy for...
9943,Wtf pangalawa si roxas #Eleksyon2016,1,wtf pangalawa si roxas
9944,Di ba pwede ang Duterte-Poe tandem?,0,di ba pwede ang duterte poe tandem
9945,Mar Roxas should now stop entertaining the pre...,1,mar roxas should now stop entertaining the pre...
9946,"#UNTVRepaso2015 Dismissed Makati Mayor Binay, ...",0,dismissed makati mayor binay sinampahan ng bag...
9947,"With 71.56% precincts reported, Mar Roxas rise...",0,with precincts reported mar roxas rises to m...
9948,Pansinin niyo naman anak ni VP Binay. :(,0,pansinin niyo naman anak ni vp binay
9949,"Kring-kring, Binay... kring-kring\r\rMore vide...",0,kring kring binay kring kringmore videos i lik...


In [25]:
#df['Another'] = df['Another'].replace('', np.nan)
#replace all empty spaces with NaN to drop using dropna
train['clean_tweet'] = train['clean_tweet'].replace('', np.NaN)

In [26]:
train.dropna(axis='rows')

,text,label,clean_tweet
0,Inaasahan na ni Vice President Jejomar Binay n...,0,inaasahan na ni vice president jejomar binay n...
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1,mar roxas tang ina tuwid na daan daw eh sya n...
2,Salamat sa walang sawang suporta ng mga taga m...,0,salamat sa walang sawang suporta ng mga taga m...
3,@rapplerdotcom putangina mo binay TAKBO PA,1,putangina mo binay takbo pa
4,"Binay with selective amnesia, forgetting about...",0,binay with selective amnesia forgetting about ...
...,...,...,...
9994,kaninang pa itong Binay binay binay.....tch,1,kaninang pa itong binay binay binaytch
9995,"Mar Roxas on the rise, w/ momentum, machinery,...",0,mar roxas on the rise w momentum machinery gr...
9997,"Alan Cayetano 'confirms' Palace, Roxas, Poe be...",0,alan cayetano confirms palace roxas poe behind...
9998,Mas nakakainis ad ni Mar kaysa kay Binay.,1,mas nakakainis ad ni mar kaysa kay binay


In [27]:
train.head(60)

,text,label,clean_tweet
0,Inaasahan na ni Vice President Jejomar Binay n...,0,inaasahan na ni vice president jejomar binay n...
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1,mar roxas tang ina tuwid na daan daw eh sya n...
2,Salamat sa walang sawang suporta ng mga taga m...,0,salamat sa walang sawang suporta ng mga taga m...
3,@rapplerdotcom putangina mo binay TAKBO PA,1,putangina mo binay takbo pa
4,"Binay with selective amnesia, forgetting about...",0,binay with selective amnesia forgetting about ...
5,It doesn't matter whoever won between Duterte ...,0,it doesnt matter whoever won between duterte &...
6,Nognog? Pero nognog din ang nag malasakit? Wtf...,1,nognog pero nognog din ang nag malasakit wtf t...
7,#OnlyB1nay ?? #FB https://t.co/QEQnsK67Gm,1,NaN
8,What Abi Binay said on running for Makati mayo...,0,what abi binay said on running for makati mayor
9,Srsly. How can Binay do away with no tax for t...,1,srsly how can binay do away with no tax for th...


In [28]:
train.tail(60)

,text,label,clean_tweet
9940,Masamang espiritu sana ang sumanib sayo! #Lune...,1,masamang espiritu sana ang sumanib sayo
9941,"""Kay Binay, gaganda ang buhay; oh oh oh oh oh....",0,kay binay gaganda ang buhay oh oh oh oh oh
9942,Please Wag Lang Poe Si Roxas! #Halalan2016 #Lo...,0,please wag lang poe si roxas feeling happy for...
9943,Wtf pangalawa si roxas #Eleksyon2016,1,wtf pangalawa si roxas
9944,Di ba pwede ang Duterte-Poe tandem?,0,di ba pwede ang duterte poe tandem
9945,Mar Roxas should now stop entertaining the pre...,1,mar roxas should now stop entertaining the pre...
9946,"#UNTVRepaso2015 Dismissed Makati Mayor Binay, ...",0,dismissed makati mayor binay sinampahan ng bag...
9947,"With 71.56% precincts reported, Mar Roxas rise...",0,with precincts reported mar roxas rises to m...
9948,Pansinin niyo naman anak ni VP Binay. :(,0,pansinin niyo naman anak ni vp binay
9949,"Kring-kring, Binay... kring-kring\r\rMore vide...",0,kring kring binay kring kringmore videos i lik...


In [29]:
#https://statisticsglobe.com/drop-rows-blank-values-from-pandas-dataframe-python
train['clean_tweet'] = train['clean_tweet'].replace('', float('NaN'), regex = True)

In [30]:
train.dropna(inplace= True)

In [31]:
train.head(60)

,text,label,clean_tweet
0,Inaasahan na ni Vice President Jejomar Binay n...,0,inaasahan na ni vice president jejomar binay n...
1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,1,mar roxas tang ina tuwid na daan daw eh sya n...
2,Salamat sa walang sawang suporta ng mga taga m...,0,salamat sa walang sawang suporta ng mga taga m...
3,@rapplerdotcom putangina mo binay TAKBO PA,1,putangina mo binay takbo pa
4,"Binay with selective amnesia, forgetting about...",0,binay with selective amnesia forgetting about ...
5,It doesn't matter whoever won between Duterte ...,0,it doesnt matter whoever won between duterte &...
6,Nognog? Pero nognog din ang nag malasakit? Wtf...,1,nognog pero nognog din ang nag malasakit wtf t...
8,What Abi Binay said on running for Makati mayo...,0,what abi binay said on running for makati mayor
9,Srsly. How can Binay do away with no tax for t...,1,srsly how can binay do away with no tax for th...
10,"Sen Grace Poe, puro ka puso. Kaya lugmok bansa...",1,sen grace poe puro ka puso kaya lugmok bansa n...


In [32]:
train.tail(60)

,text,label,clean_tweet
9933,"Binay, Pork and Corruption: Binay PDAF Operati...",1,binay pork and corruption binay pdaf operations
9934,"On a scale from 1 to Mar Roxas, how desperate ...",1,on a scale from to mar roxas how desperate are...
9935,Puro ka nognog....kayumanggi kaya mga Pilipino...,0,puro ka nognogkayumanggi kaya mga pilipino ano...
9936,Mar Roxas gave fans 500 pesos for them to vote...,1,mar roxas gave fans pesos for them to vote him...
9938,Binubully na naman nila si Binay ??,0,binubully na naman nila si binay
9939,@Dalesanity10 last year pa to pero happy birth...,0,last year pa to pero happy birthday binay manl...
9940,Masamang espiritu sana ang sumanib sayo! #Lune...,1,masamang espiritu sana ang sumanib sayo
9941,"""Kay Binay, gaganda ang buhay; oh oh oh oh oh....",0,kay binay gaganda ang buhay oh oh oh oh oh
9942,Please Wag Lang Poe Si Roxas! #Halalan2016 #Lo...,0,please wag lang poe si roxas feeling happy for...
9943,Wtf pangalawa si roxas #Eleksyon2016,1,wtf pangalawa si roxas


In [34]:
first_column = train.pop('label')
train.insert(0,'label',first_column)

In [35]:
train.head(10)

,label,text,clean_tweet
0,0,Inaasahan na ni Vice President Jejomar Binay n...,inaasahan na ni vice president jejomar binay n...
1,1,Mar Roxas TANG INA TUWID NA DAAN DAW .. EH SYA...,mar roxas tang ina tuwid na daan daw eh sya n...
2,0,Salamat sa walang sawang suporta ng mga taga m...,salamat sa walang sawang suporta ng mga taga m...
3,1,@rapplerdotcom putangina mo binay TAKBO PA,putangina mo binay takbo pa
4,0,"Binay with selective amnesia, forgetting about...",binay with selective amnesia forgetting about ...
5,0,It doesn't matter whoever won between Duterte ...,it doesnt matter whoever won between duterte &...
6,1,Nognog? Pero nognog din ang nag malasakit? Wtf...,nognog pero nognog din ang nag malasakit wtf t...
8,0,What Abi Binay said on running for Makati mayo...,what abi binay said on running for makati mayor
9,1,Srsly. How can Binay do away with no tax for t...,srsly how can binay do away with no tax for th...
10,1,"Sen Grace Poe, puro ka puso. Kaya lugmok bansa...",sen grace poe puro ka puso kaya lugmok bansa n...


In [36]:
#total data entries for training

print(train.shape)

(9482, 3)


In [37]:
# remove unwanted text from test data
test.head(10)

,text,label
0,Unshaded votes and votes for Mayor Duterte goe...,1
1,#NoMoreChance https://t.co/msaaUGv0bS,1
2,@itsmanj well there's other good choices like ...,0
3,Nognog. Pandak. Laki sa hirap. Pero corrupt. Y...,1
4,Ex-Binay aide turns tables on Mercado | https:...,0
5,2015/10/15 11:37 PHT,0
6,"Binay in Isabela radio interview: ‘Yong iba, ...",0
7,Poor Binay ?? https://t.co/In9yRo5QXI,1
8,RT @sizzlingbubbles: Juskopo! Gugustuhin kong ...,1
9,"ISA LANG MASASABI KO SA AD NI BINAY, P U T A. ...",1


In [38]:
first_column = test.pop('label')
test.insert(0,'label',first_column)

In [39]:
test.head(10)

,label,text
0,1,Unshaded votes and votes for Mayor Duterte goe...
1,1,#NoMoreChance https://t.co/msaaUGv0bS
2,0,@itsmanj well there's other good choices like ...
3,1,Nognog. Pandak. Laki sa hirap. Pero corrupt. Y...
4,0,Ex-Binay aide turns tables on Mercado | https:...
5,0,2015/10/15 11:37 PHT
6,0,"Binay in Isabela radio interview: ‘Yong iba, ..."
7,1,Poor Binay ?? https://t.co/In9yRo5QXI
8,1,RT @sizzlingbubbles: Juskopo! Gugustuhin kong ...
9,1,"ISA LANG MASASABI KO SA AD NI BINAY, P U T A. ..."


In [40]:
test_tweet = clean_tweets(test["text"])
test_tweet = pd.DataFrame(test_tweet)

In [41]:
# append cleaned tweets to the training data
test["clean_tweet"] = test_tweet

# compare the cleaned and uncleaned tweets
test.head(50)

,label,text,clean_tweet
0,1,Unshaded votes and votes for Mayor Duterte goe...,unshaded votes and votes for mayor duterte goe...
1,1,#NoMoreChance https://t.co/msaaUGv0bS,
2,0,@itsmanj well there's other good choices like ...,well theres other good choices like duterte or...
3,1,Nognog. Pandak. Laki sa hirap. Pero corrupt. Y...,nognog pandak laki sa hirap pero corrupt yan s...
4,0,Ex-Binay aide turns tables on Mercado | https:...,ex binay aide turns tables on mercado
5,0,2015/10/15 11:37 PHT,10 15 37 pht
6,0,"Binay in Isabela radio interview: ‘Yong iba, ...",binay in isabela radio interview yong iba nang...
7,1,Poor Binay ?? https://t.co/In9yRo5QXI,poor binay
8,1,RT @sizzlingbubbles: Juskopo! Gugustuhin kong ...,juskopo gugustuhin kong ma clark ma leah pwed...
9,1,"ISA LANG MASASABI KO SA AD NI BINAY, P U T A. ...",isa lang masasabi ko sa ad ni binay p u t a bo...


In [42]:
#https://statisticsglobe.com/drop-rows-blank-values-from-pandas-dataframe-python
test['clean_tweet'] = test['clean_tweet'].replace('', float('NaN'), regex = True)
test.dropna(inplace= True)


In [43]:
test.head(60)

,label,text,clean_tweet
0,1,Unshaded votes and votes for Mayor Duterte goe...,unshaded votes and votes for mayor duterte goe...
2,0,@itsmanj well there's other good choices like ...,well theres other good choices like duterte or...
3,1,Nognog. Pandak. Laki sa hirap. Pero corrupt. Y...,nognog pandak laki sa hirap pero corrupt yan s...
4,0,Ex-Binay aide turns tables on Mercado | https:...,ex binay aide turns tables on mercado
5,0,2015/10/15 11:37 PHT,10 15 37 pht
6,0,"Binay in Isabela radio interview: ‘Yong iba, ...",binay in isabela radio interview yong iba nang...
7,1,Poor Binay ?? https://t.co/In9yRo5QXI,poor binay
8,1,RT @sizzlingbubbles: Juskopo! Gugustuhin kong ...,juskopo gugustuhin kong ma clark ma leah pwed...
9,1,"ISA LANG MASASABI KO SA AD NI BINAY, P U T A. ...",isa lang masasabi ko sa ad ni binay p u t a bo...
10,0,#OnlyBinayWinner Only B1NAY,only b1nay


In [44]:
#remove stopwords
import urllib.request, json 
with urllib.request.urlopen("https://raw.githubusercontent.com/stopwords-iso/stopwords-tl/master/stopwords-tl.json") as url:
    stopwords = json.loads(url.read().decode())
    print(stopwords)

['akin', 'aking', 'ako', 'alin', 'am', 'amin', 'aming', 'ang', 'ano', 'anumang', 'apat', 'at', 'atin', 'ating', 'ay', 'bababa', 'bago', 'bakit', 'bawat', 'bilang', 'dahil', 'dalawa', 'dapat', 'din', 'dito', 'doon', 'gagawin', 'gayunman', 'ginagawa', 'ginawa', 'ginawang', 'gumawa', 'gusto', 'habang', 'hanggang', 'hindi', 'huwag', 'iba', 'ibaba', 'ibabaw', 'ibig', 'ikaw', 'ilagay', 'ilalim', 'ilan', 'inyong', 'isa', 'isang', 'itaas', 'ito', 'iyo', 'iyon', 'iyong', 'ka', 'kahit', 'kailangan', 'kailanman', 'kami', 'kanila', 'kanilang', 'kanino', 'kanya', 'kanyang', 'kapag', 'kapwa', 'karamihan', 'katiyakan', 'katulad', 'kaya', 'kaysa', 'ko', 'kong', 'kulang', 'kumuha', 'kung', 'laban', 'lahat', 'lamang', 'likod', 'lima', 'maaari', 'maaaring', 'maging', 'mahusay', 'makita', 'marami', 'marapat', 'masyado', 'may', 'mayroon', 'mga', 'minsan', 'mismo', 'mula', 'muli', 'na', 'nabanggit', 'naging', 'nagkaroon', 'nais', 'nakita', 'namin', 'napaka', 'narito', 'nasaan', 'ng', 'ngayon', 'ni', 'nila',

In [63]:
#data['content2'] =data['Content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
train['rm_stpwrds'] = train['clean_tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords) ]))

In [64]:
train[['clean_tweet', 'rm_stpwrds']].head(50)

,clean_tweet,rm_stpwrds
0,inaasahan na ni vice president jejomar binay n...,inaasahan vice president jejomar binay taong
1,mar roxas tang ina tuwid na daan daw eh sya n...,mar roxas tang ina tuwid daan daw eh sya nga d...
2,salamat sa walang sawang suporta ng mga taga m...,salamat sawang suporta taga makati pagbabalik ...
3,putangina mo binay takbo pa,putangina mo binay takbo
4,binay with selective amnesia forgetting about ...,binay with selective amnesia forgetting about ...
5,it doesnt matter whoever won between duterte &...,it doesnt matter whoever won between duterte &...
6,nognog pero nognog din ang nag malasakit wtf t...,nognog nognog nag malasakit wtf tangina mo bin...
8,what abi binay said on running for makati mayor,what abi binay said on running for makati mayor
9,srsly how can binay do away with no tax for th...,srsly how can binay do away with no tax for th...
10,sen grace poe puro ka puso kaya lugmok bansa n...,sen grace poe puro puso lugmok bansa natin kah...


In [67]:
#tokenization 

import nltk
nltk.download('punkt')
from nltk.tokenize import WhitespaceTokenizer
train['tokenize'] = train['rm_stpwrds'].apply(nltk.tokenize.WhitespaceTokenizer().tokenize) 
train['tokenize'].head()



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


0    [inaasahan, vice, president, jejomar, binay, t...
1    [mar, roxas, tang, ina, tuwid, daan, daw, eh, ...
2    [salamat, sawang, suporta, taga, makati, pagba...
3                        [putangina, mo, binay, takbo]
4    [binay, with, selective, amnesia, forgetting, ...
Name: tokenize, dtype: object

In [72]:
train[['rm_stpwrds','tokenize']].head(30)

,rm_stpwrds,tokenize
0,inaasahan vice president jejomar binay taong,"[inaasahan, vice, president, jejomar, binay, t..."
1,mar roxas tang ina tuwid daan daw eh sya nga d...,"[mar, roxas, tang, ina, tuwid, daan, daw, eh, ..."
2,salamat sawang suporta taga makati pagbabalik ...,"[salamat, sawang, suporta, taga, makati, pagba..."
3,putangina mo binay takbo,"[putangina, mo, binay, takbo]"
4,binay with selective amnesia forgetting about ...,"[binay, with, selective, amnesia, forgetting, ..."
5,it doesnt matter whoever won between duterte &...,"[it, doesnt, matter, whoever, won, between, du..."
6,nognog nognog nag malasakit wtf tangina mo bin...,"[nognog, nognog, nag, malasakit, wtf, tangina,..."
8,what abi binay said on running for makati mayor,"[what, abi, binay, said, on, running, for, mak..."
9,srsly how can binay do away with no tax for th...,"[srsly, how, can, binay, do, away, with, no, t..."
10,sen grace poe puro puso lugmok bansa natin kah...,"[sen, grace, poe, puro, puso, lugmok, bansa, n..."


# Decided not to proceed with Lemmetization and Stemming because nltk does not support the Filipino Language, those not having any significant impact towards the data.